<a href="https://colab.research.google.com/github/stayup24h/Hangul-to-Unicode-Obfuscation-Project/blob/main/unicode_similarity_search_by_OCR_vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from sklearn.model_selection import KFold
from copy import deepcopy # 각 폴드마다 새로운 모델을 시작하기 위함

In [5]:
# autoencoder 정의

def compiled_autoencoder(input_dim=68, latent_dim=16):
    """Autoencoder 모델을 정의하고 컴파일하여 반환합니다."""

    # 1. Encoder 정의
    input_layer = tf.keras.layers.Input(shape=(input_dim,), name='input_vector')
    x = tf.keras.layers.Dense(32, activation='relu')(input_layer)
    latent_vector = tf.keras.layers.Dense(latent_dim, activation='relu', name='latent_vector')(x)
    encoder = Model(input_layer, latent_vector, name='encoder')

    # 2. Decoder 정의
    latent_input = tf.keras.layers.Input(shape=(latent_dim,), name='latent_input')
    y = tf.keras.layers.Dense(32, activation='relu')(latent_input)
    reconstruction = tf.keras.layers.Dense(input_dim, activation='sigmoid', name='reconstruction')(y)
    decoder = Model(latent_input, reconstruction, name='decoder')

    # 3. Autoencoder 정의
    autoencoder_output = decoder(encoder(input_layer))
    autoencoder = Model(input_layer, autoencoder_output, name='autoencoder')

    # 4. 컴파일
    autoencoder.compile(optimizer='adam', loss='mse')

    return autoencoder, encoder

In [ ]:
# autoencoder 학습

# TODO : 유니코드를 OCR한 68차원 numpy 배열 불러오기

N = 200000 # numpy 배열 크기
INPUT_DIM = 68
X_features_all = np.random.rand(N, INPUT_DIM).astype('float32')
LATENT_DIM = 16

K_FOLDS = 10
kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

EPOCHS = 20
BATCH_SIZE = 256

fold_results = []
trained_encoders = []
best_loss = float('inf')
best_encoder = None

print(f"Starting {K_FOLDS}-Fold Cross-Validation")

for fold, (train_index, val_index) in enumerate(kf.split(X_features_all)):
    print(f"\n--- Fold {fold+1}/{K_FOLDS} ---")

    X_train, X_val = X_features_all[train_index], X_features_all[val_index]

    autoencoder, current_encoder = compiled_autoencoder(INPUT_DIM, LATENT_DIM)

    history = autoencoder.fit(
        x=X_train,
        y=X_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_val, X_val),
        verbose=1
    )

    val_loss = history.history['val_loss'][-1]
    fold_results.append(val_loss)
    print(f"Fold {fold+1} Validation Loss: {val_loss:.6f}")

    if val_loss < best_loss:
        best_loss = val_loss
        best_encoder = deepcopy(current_encoder)

mean_val_loss = np.mean(fold_results)
std_val_loss = np.std(fold_results)

print("\n--- K-Fold Summary ---")
print(f"Individual Fold Losses: {fold_results}")
print(f"Mean Validation Loss across {K_FOLDS} Folds: {mean_val_loss:.6f} (+/- {std_val_loss:.6f})")

if best_encoder:
    best_encoder.save('best_kf_encoder_for_faiss.h5')
    print("\nBest Encoder Model saved successfully!")

In [ ]:
# 68차원 벡터들을 16차원 벡터들로 변환

best_encoder = load_model('best_kf_encoder_for_faiss.h5')

# TODO : 유니코드를 OCR한 68차원 numpy 배열 불러오기

# X_features_all = np.load('all_unicode_features.npy')
N = 200000 #numpy 배열 크기
INPUT_DIM = 68
X_features_all = np.random.rand(N, INPUT_DIM).astype('float32') # 예시 데이터

X_16dim_vectors = best_encoder.predict(X_features_all)

X_16dim_vectors = X_16dim_vectors.astype('float32')

In [ ]:
# 변환한 벡터들 저장

VECTOR_OUTPUT_FILENAME = "faiss_16dim_vectors.npy"

np.save(VECTOR_OUTPUT_FILENAME, X_16dim_vectors)

print(f"16차원 잠재 벡터가 {VECTOR_OUTPUT_FILENAME}에 저장되었습니다. (Shape: {X_16dim_vectors.shape})")

In [ ]:
# 유니코드 코드 포인트 배열 생성 (N개의 요소)
# 이 배열은 X_features_all을 생성했을 때의 순서와 정확히 일치해야 합니다.
# 예시: 한글 외 유니코드 코드 포인트만 순서대로 나열
# [0x0000, 0x0001, ..., 0x00A0, ..., 0x10FFFF] (제외 범위 제외)
unicode_codepoints = np.array([
    # 실제 데이터셋을 만들 때 사용된 코드 포인트를 순서대로 여기에 배치해야 함.
    i for i in range(N) # 예시에서는 N개의 임의 인덱스라고 가정
])

MAPPING_OUTPUT_FILENAME = "faiss_unicode_map.npy"
np.save(MAPPING_OUTPUT_FILENAME, unicode_codepoints)

print(f"유니코드 코드 포인트 매핑이 {MAPPING_OUTPUT_FILENAME}에 저장되었습니다. (Shape: {unicode_codepoints.shape})")